<a href="https://colab.research.google.com/github/Mithil01/Human-Activity-Recognition/blob/main/HumanActivityRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing Libraries

In [61]:
!pip install prettytable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!unzip train.zip
!unzip test.zip

Archive:  train.zip
  inflating: train/.DS_Store         
   creating: train/Inertial Signals/
  inflating: train/Inertial Signals/body_acc_x_train.txt  
  inflating: train/Inertial Signals/body_acc_y_train.txt  
  inflating: train/Inertial Signals/body_acc_z_train.txt  
  inflating: train/Inertial Signals/body_gyro_x_train.txt  
  inflating: train/Inertial Signals/body_gyro_y_train.txt  
  inflating: train/Inertial Signals/body_gyro_z_train.txt  
  inflating: train/Inertial Signals/total_acc_x_train.txt  
  inflating: train/Inertial Signals/total_acc_y_train.txt  
  inflating: train/Inertial Signals/total_acc_z_train.txt  
  inflating: train/subject_train.txt  
  inflating: train/X_train.txt       
  inflating: train/y_train.txt       
Archive:  test.zip
   creating: test/Inertial Signals/
  inflating: test/Inertial Signals/body_acc_x_test.txt  
  inflating: test/Inertial Signals/body_acc_y_test.txt  
  inflating: test/Inertial Signals/body_acc_z_test.txt  
  inflating: test/Inertial 

In [62]:
import pandas as pd
import numpy as np

In [1]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [23]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []
    
    for signal in SIGNALS:
        filename = f'{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 
      

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [87]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    """
    filename = f'{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]


    return pd.get_dummies(y).to_numpy()

In [20]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [28]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf


In [32]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [33]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [34]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [24]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [26]:
X_train.shape

(7352, 128, 9)

In [45]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)
n_outputs = Y_train.shape[1]

print(timesteps)
print(input_dim)
print(len(X_train))
print(n_outputs)

128
9
7352
6


- Defining the Architecture of LSTM

In [36]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [38]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 27s 53ms/step - loss: 1.2464 - accuracy: 0.4894 - val_loss: 1.0256 - val_accuracy: 0.5914
Epoch 2/30
460/460 [==============================] - 25s 54ms/step - loss: 0.8947 - accuracy: 0.6190 - val_loss: 0.8414 - val_accuracy: 0.6396
Epoch 3/30
460/460 [==============================] - 24s 53ms/step - loss: 0.7524 - accuracy: 0.6665 - val_loss: 0.7585 - val_accuracy: 0.7109
Epoch 4/30
460/460 [==============================] - 25s 55ms/step - loss: 0.6359 - accuracy: 0.7372 - val_loss: 0.7886 - val_accuracy: 0.6871
Epoch 5/30
460/460 [==============================] - 23s 49ms/step - loss: 0.5612 - accuracy: 0.7866 - val_loss: 0.6080 - val_accuracy: 0.7737
Epoch 6/30
460/460 [==============================] - 22s 47ms/step - loss: 0.4705 - accuracy: 0.8443 - val_loss: 0.5059 - val_accuracy: 0.8168
Epoch 7/30
460/460 [==============================] - 24s 52ms/step - loss: 0.3568 - accuracy: 0.8904 - val_loss: 0.4619 - val_accuracy:

In [39]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

93/93 [==============================] - 2s 12ms/step
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      409        77        1                   0   
STANDING                 0      118       413        1                   0   
WALKING                  0        0         2      488                   4   
WALKING_DOWNSTAIRS       0        0         0        3                 412   
WALKING_UPSTAIRS         0        0         0       40                   8   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            4  
STANDING                           0  
WALKING                            2  
WALKING_DOWNSTAIRS                 5  
WALKING_UPSTAIRS                 423  


In [40]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 2s 20ms/step - loss: 0.4038 - accuracy: 0.9009


In [ ]:
score

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.40
- We can further imporve the performace with Hyperparameter tuning

# **Now we will see how does 1D convolution perform for this task and compare its results with LSTM results.**







In [43]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten

In [85]:
#Defining the arch of Convolution model
convmodel = Sequential()
convmodel.add(Conv1D(32,3,activation = 'relu', input_shape=(timesteps, input_dim)))
convmodel.add(Conv1D(32,3,activation = 'relu'))
convmodel.add(Dropout(0.5))
convmodel.add(MaxPooling1D(pool_size=2))
convmodel.add(Flatten())
convmodel.add(Dense(64,activation='relu'))
convmodel.add(Dense(n_outputs, activation='softmax'))
convmodel.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 126, 32)           896       
                                                                 
 conv1d_9 (Conv1D)           (None, 124, 32)           3104      
                                                                 
 dropout_3 (Dropout)         (None, 124, 32)           0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 62, 32)           0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 1984)              0         
                                                                 
 dense_5 (Dense)             (None, 64)                127040    
                                                      

In [51]:
# Compiling the Convolution model
convmodel.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [52]:
# Training the convolution model
convmodel.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 7s 14ms/step - loss: 0.4017 - accuracy: 0.8406 - val_loss: 0.3236 - val_accuracy: 0.8904
Epoch 2/30
460/460 [==============================] - 7s 15ms/step - loss: 0.1428 - accuracy: 0.9441 - val_loss: 0.3564 - val_accuracy: 0.8989
Epoch 3/30
460/460 [==============================] - 6s 13ms/step - loss: 0.1271 - accuracy: 0.9474 - val_loss: 0.2701 - val_accuracy: 0.8894
Epoch 4/30
460/460 [==============================] - 6s 13ms/step - loss: 0.1083 - accuracy: 0.9565 - val_loss: 0.3512 - val_accuracy: 0.8687
Epoch 5/30
460/460 [==============================] - 7s 15ms/step - loss: 0.1196 - accuracy: 0.9499 - val_loss: 0.2353 - val_accuracy: 0.9152
Epoch 6/30
460/460 [==============================] - 7s 15ms/step - loss: 0.1041 - accuracy: 0.9566 - val_loss: 0.2820 - val_accuracy: 0.9053
Epoch 7/30
460/460 [==============================] - 6s 13ms/step - loss: 0.1067 - accuracy: 0.9513 - val_loss: 0.2669 - val_accuracy: 0.9162

In [53]:
# Confusion Matrix for convolution model
print(confusion_matrix(Y_test, convmodel.predict(X_test)))

93/93 [==============================] - 1s 6ms/step
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        22        0                   0   
SITTING                  0      381       105        0                   0   
STANDING                 0       58       469        1                   0   
WALKING                  0        0         0      485                  11   
WALKING_DOWNSTAIRS       0        0         0       10                 406   
WALKING_UPSTAIRS         0        0         0       25                  23   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             5  
SITTING                            5  
STANDING                           4  
WALKING                            0  
WALKING_DOWNSTAIRS                 4  
WALKING_UPSTAIRS                 423  


In [54]:
convscore = convmodel.evaluate(X_test, Y_test)

93/93 [==============================] - 1s 9ms/step - loss: 0.5346 - accuracy: 0.9074


In [55]:
convscore 

[0.534570574760437, 0.9073634147644043]

In [63]:
from prettytable import PrettyTable

In [81]:
conv_loss, conv_acc = np.around(convscore,decimals=3)
lstm_loss, lstm_acc = np.around(score,decimals=3)

In [86]:
myTable = PrettyTable(["Model", "Accuracy", "Loss"], digits = 3, round = True)
myTable.add_row(["LSTM ",lstm_acc, lstm_loss])
myTable.add_row(["Convolution1D ",conv_acc, conv_loss])
print(myTable)

+----------------+----------+-------+
|     Model      | Accuracy |  Loss |
+----------------+----------+-------+
|     LSTM       |  0.901   | 0.404 |
| Convolution1D  |  0.907   | 0.535 |
+----------------+----------+-------+


# As we can see from above results of the LSTM and Convolution model for Human Activity Recognition which is multi-class classification task - accuracy obtained from the both models are almost same for same number of epochs although 2 layer of convolution is used while in the case of LSTM its single layer.